# gru 生成模型 

参照官方文档    
https://tensorflow.google.cn/tutorials/text/text_generation

## 1数据处理

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
# 读取并为 py2 compat 解码
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
text = text[:111539]
# 文本长度是指文本中的字符个数
print ('Length of text: {} characters'.format(len(text)))
text[:30]

Length of text: 111539 characters


'First Citizen:\nBefore we proce'

In [3]:
# 文本中的非重复字符
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))


61 unique characters


In [4]:
# 创建从非重复字符到索引的映射
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])
text_as_int.shape

(111539,)

In [5]:
# 设定每个输入句子长度的最大值
seq_length = 100
examples_per_epoch = len(text)//seq_length
print("examples_per_epoch",examples_per_epoch)

# 创建训练样本 / 目标
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
print("char_dataset",char_dataset)

for i in char_dataset.take(5):
    print(i)
    print(idx2char[i])


examples_per_epoch 1115
char_dataset <TensorSliceDataset shapes: (), types: tf.int32>
tf.Tensor(16, shape=(), dtype=int32)
F
tf.Tensor(43, shape=(), dtype=int32)
i
tf.Tensor(52, shape=(), dtype=int32)
r
tf.Tensor(53, shape=(), dtype=int32)
s
tf.Tensor(54, shape=(), dtype=int32)
t


In [6]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)#drop_remainder 最后省的尾巴还要不要

for item in sequences.take(5):
#     print(item)
    print(repr(''.join(idx2char[item])))


'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [7]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))
dataset

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


<MapDataset shapes: ((100,), (100,)), types: (tf.int32, tf.int32)>

In [8]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))


Step    0
  input: 16 ('F')
  expected output: 43 ('i')
Step    1
  input: 43 ('i')
  expected output: 52 ('r')
Step    2
  input: 52 ('r')
  expected output: 53 ('s')
Step    3
  input: 53 ('s')
  expected output: 54 ('t')
Step    4
  input: 54 ('t')
  expected output: 1 (' ')


In [9]:
# 批大小
BATCH_SIZE = 64

# 设定缓冲区大小，以重新排列数据集
# （TF 数据被设计为可以处理可能是无限的序列，
# 所以它不会试图在内存中重新排列整个序列。相反，
# 它维持一个缓冲区，在缓冲区重新排列元素。） 
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

## 2模型构建

In [10]:
# 词集的长度
vocab_size = len(vocab)

# 嵌入的维度
embedding_dim = 256

# RNN 的单元数量
rnn_units = 1024


In [11]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model


In [12]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)


In [13]:
for input_example_batch, target_example_batch in dataset.take(1):
    print(input_example_batch)
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


tf.Tensor(
[[55 46  0 ... 55  5  0]
 [54 42  1 ... 38 49  1]
 [46 40  1 ... 35  1 46]
 ...
 [48 41 46 ...  1 59 49]
 [ 1 40 49 ...  4 53  1]
 [ 5  0 30 ... 42 59  1]], shape=(64, 100), dtype=int32)
(64, 100, 61) # (batch_size, sequence_length, vocab_size)


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           15616     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 61)            62525     
Total params: 4,016,445
Trainable params: 4,016,445
Non-trainable params: 0
_________________________________________________________________


In [15]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())


Prediction shape:  (64, 100, 61)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1107197


In [16]:
model.compile(optimizer='adam', loss=loss)



In [17]:
# 检查点保存至的目录
checkpoint_dir = './training_checkpoints/gru_ssby/'

# 检查点的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)



In [37]:
history = model.fit(dataset, epochs=10, callbacks=[checkpoint_callback])


Epoch 1/10
17/17 [==============================] - 42s 2s/step - loss: 4.1401
Epoch 2/10
17/17 [==============================] - 42s 2s/step - loss: 3.2835
Epoch 3/10
17/17 [==============================] - 41s 2s/step - loss: 2.8464
Epoch 4/10
17/17 [==============================] - 41s 2s/step - loss: 2.5419
Epoch 5/10
17/17 [==============================] - 41s 2s/step - loss: 2.3731
Epoch 6/10
17/17 [==============================] - 42s 2s/step - loss: 2.2738
Epoch 7/10
17/17 [==============================] - 43s 3s/step - loss: 2.2032
Epoch 8/10
17/17 [==============================] - 43s 3s/step - loss: 2.1391
Epoch 9/10
17/17 [==============================] - 43s 3s/step - loss: 2.0808
Epoch 10/10
17/17 [==============================] - 44s 3s/step - loss: 2.0231


## 3文本生成

In [38]:
tf.train.latest_checkpoint(checkpoint_dir)


'./training_checkpoints/gru_ssby/ckpt_10'

In [39]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            15616     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_2 (Dense)              (1, None, 61)             62525     
Total params: 4,016,445
Trainable params: 4,016,445
Non-trainable params: 0
_________________________________________________________________


In [49]:
# a=[1,2,3]
# a=np.array(a)
# a=np.reshape(a,[1,-1,1])
# a=a.flatten()
# a

In [56]:
def generate_text(model, start_string):
  # 评估步骤（用学习过的模型生成文本）

  # 要生成的字符个数
    num_generate = 1000

  # 将起始字符串转换为数字（向量化）
    input_eval = [char2idx[s] for s in start_string]
    print(input_eval)
    input_eval = tf.expand_dims(input_eval, 0)
    print(input_eval)

  # 空字符串用于存储结果
    text_generated = []

  # 低温度会生成更可预测的文本
  # 较高温度会生成更令人惊讶的文本
  # 可以通过试验以找到最好的设定
    temperature = 1.0

  # 这里批大小为 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        print(predictions.shape)
      # 删除批次的维度
        predictions = tf.squeeze(predictions, 0)
        print(predictions.shape)
        
      # 用分类分布预测模型返回的字符
        predictions = predictions / temperature
#         print(predictions)
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        print(predicted_id)

      # 把预测字符和前面的隐藏状态一起传递给模型作为下一个输入
        input_eval = tf.expand_dims([predicted_id], 0)
        print(input_eval.shape)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))


In [57]:
print(generate_text(model, start_string=u"ask "))


[35, 53, 45, 1]
tf.Tensor([[35 53 45  1]], shape=(1, 4), dtype=int32)
(1, 4, 61)
(4, 61)
49
(1, 1)
(1, 1, 61)
(1, 61)
40
(1, 1)
(1, 1, 61)
(1, 61)
1
(1, 1)
(1, 1, 61)
(1, 61)
59
(1, 1)
(1, 1, 61)
(1, 61)
49
(1, 1)
(1, 1, 61)
(1, 61)
54
(1, 1)
(1, 1, 61)
(1, 61)
10
(1, 1)
(1, 1, 61)
(1, 61)
0
(1, 1)
(1, 1, 61)
(1, 61)
0
(1, 1)
(1, 1, 61)
(1, 61)
11
(1, 1)
(1, 1, 61)
(1, 61)
46
(1, 1)
(1, 1, 61)
(1, 61)
46
(1, 1)
(1, 1, 61)
(1, 61)
5
(1, 1)
(1, 1, 61)
(1, 61)
0
(1, 1)
(1, 1, 61)
(1, 61)
30
(1, 1)
(1, 1, 61)
(1, 61)
42
(1, 1)
(1, 1, 61)
(1, 61)
1
(1, 1)
(1, 1, 61)
(1, 61)
54
(1, 1)
(1, 1, 61)
(1, 61)
42
(1, 1)
(1, 1, 61)
(1, 61)
43
(1, 1)
(1, 1, 61)
(1, 61)
54
(1, 1)
(1, 1, 61)
(1, 61)
42
(1, 1)
(1, 1, 61)
(1, 61)
1
(1, 1)
(1, 1, 61)
(1, 61)
37
(1, 1)
(1, 1, 61)
(1, 61)
49
(1, 1)
(1, 1, 61)
(1, 61)
46
(1, 1)
(1, 1, 61)
(1, 61)
37
(1, 1)
(1, 1, 61)
(1, 61)
55
(1, 1)
(1, 1, 61)
(1, 61)
53
(1, 1)
(1, 1, 61)
(1, 61)
8
(1, 1)
(1, 1, 61)
(1, 61)
0
(1, 1)
(1, 1, 61)
(1, 61)
25
(1, 1)
(1, 1, 61)


(1, 61)
45
(1, 1)
(1, 1, 61)
(1, 61)
39
(1, 1)
(1, 1, 61)
(1, 61)
1
(1, 1)
(1, 1, 61)
(1, 61)
43
(1, 1)
(1, 1, 61)
(1, 61)
40
(1, 1)
(1, 1, 61)
(1, 61)
57
(1, 1)
(1, 1, 61)
(1, 61)
43
(1, 1)
(1, 1, 61)
(1, 61)
54
(1, 1)
(1, 1, 61)
(1, 61)
42
(1, 1)
(1, 1, 61)
(1, 61)
1
(1, 1)
(1, 1, 61)
(1, 61)
40
(1, 1)
(1, 1, 61)
(1, 61)
49
(1, 1)
(1, 1, 61)
(1, 61)
47
(1, 1)
(1, 1, 61)
(1, 61)
39
(1, 1)
(1, 1, 61)
(1, 61)
0
(1, 1)
(1, 1, 61)
(1, 61)
33
(1, 1)
(1, 1, 61)
(1, 61)
49
(1, 1)
(1, 1, 61)
(1, 61)
49
(1, 1)
(1, 1, 61)
(1, 61)
46
(1, 1)
(1, 1, 61)
(1, 61)
38
(1, 1)
(1, 1, 61)
(1, 61)
1
(1, 1)
(1, 1, 61)
(1, 61)
53
(1, 1)
(1, 1, 61)
(1, 61)
57
(1, 1)
(1, 1, 61)
(1, 61)
42
(1, 1)
(1, 1, 61)
(1, 61)
35
(1, 1)
(1, 1, 61)
(1, 61)
52
(1, 1)
(1, 1, 61)
(1, 61)
45
(1, 1)
(1, 1, 61)
(1, 61)
1
(1, 1)
(1, 1, 61)
(1, 61)
38
(1, 1)
(1, 1, 61)
(1, 61)
43
(1, 1)
(1, 1, 61)
(1, 61)
54
(1, 1)
(1, 1, 61)
(1, 61)
1
(1, 1)
(1, 1, 61)
(1, 61)
49
(1, 1)
(1, 1, 61)
(1, 61)
47
(1, 1)
(1, 1, 61)
(1, 61)
39
(1, 1)
(1

(1, 61)
48
(1, 1)
(1, 1, 61)
(1, 61)
1
(1, 1)
(1, 1, 61)
(1, 61)
54
(1, 1)
(1, 1, 61)
(1, 61)
49
(1, 1)
(1, 1, 61)
(1, 61)
48
(1, 1)
(1, 1, 61)
(1, 61)
54
(1, 1)
(1, 1, 61)
(1, 61)
1
(1, 1)
(1, 1, 61)
(1, 61)
54
(1, 1)
(1, 1, 61)
(1, 61)
49
(1, 1)
(1, 1, 61)
(1, 61)
49
(1, 1)
(1, 1, 61)
(1, 61)
48
(1, 1)
(1, 1, 61)
(1, 61)
55
(1, 1)
(1, 1, 61)
(1, 61)
52
(1, 1)
(1, 1, 61)
(1, 61)
1
(1, 1)
(1, 1, 61)
(1, 61)
35
(1, 1)
(1, 1, 61)
(1, 61)
54
(1, 1)
(1, 1, 61)
(1, 61)
42
(1, 1)
(1, 1, 61)
(1, 61)
53
(1, 1)
(1, 1, 61)
(1, 61)
39
(1, 1)
(1, 1, 61)
(1, 61)
48
(1, 1)
(1, 1, 61)
(1, 61)
40
(1, 1)
(1, 1, 61)
(1, 61)
1
(1, 1)
(1, 1, 61)
(1, 61)
49
(1, 1)
(1, 1, 61)
(1, 61)
50
(1, 1)
(1, 1, 61)
(1, 61)
39
(1, 1)
(1, 1, 61)
(1, 61)
1
(1, 1)
(1, 1, 61)
(1, 61)
35
(1, 1)
(1, 1, 61)
(1, 61)
53
(1, 1)
(1, 1, 61)
(1, 61)
8
(1, 1)
(1, 1, 61)
(1, 61)
0
(1, 1)
(1, 1, 61)
(1, 61)
25
(1, 1)
(1, 1, 61)
(1, 61)
42
(1, 1)
(1, 1, 61)
(1, 61)
39
(1, 1)
(1, 1, 61)
(1, 61)
1
(1, 1)
(1, 1, 61)
(1, 61)
54
(1, 1)
(1, 

(1, 61)
39
(1, 1)
(1, 1, 61)
(1, 61)
48
(1, 1)
(1, 1, 61)
(1, 61)
40
(1, 1)
(1, 1, 61)
(1, 61)
39
(1, 1)
(1, 1, 61)
(1, 61)
52
(1, 1)
(1, 1, 61)
(1, 61)
47
(1, 1)
(1, 1, 61)
(1, 61)
47
(1, 1)
(1, 1, 61)
(1, 61)
59
(1, 1)
(1, 1, 61)
(1, 61)
5
(1, 1)
(1, 1, 61)
(1, 61)
1
(1, 1)
(1, 1, 61)
(1, 61)
54
(1, 1)
(1, 1, 61)
(1, 61)
52
(1, 1)
(1, 1, 61)
(1, 61)
39
(1, 1)
(1, 1, 61)
(1, 61)
37
(1, 1)
(1, 1, 61)
(1, 61)
42
(1, 1)
(1, 1, 61)
(1, 61)
55
(1, 1)
(1, 1, 61)
(1, 61)
53
(1, 1)
(1, 1, 61)
(1, 61)
5
(1, 1)
(1, 1, 61)
(1, 61)
0
(1, 1)
(1, 1, 61)
(1, 61)
18
(1, 1)
(1, 1, 61)
(1, 61)
39
(1, 1)
(1, 1, 61)
(1, 61)
38
(1, 1)
(1, 1, 61)
(1, 61)
1
(1, 1)
(1, 1, 61)
(1, 61)
53
(1, 1)
(1, 1, 61)
(1, 61)
43
(1, 1)
(1, 1, 61)
(1, 61)
50
(1, 1)
(1, 1, 61)
(1, 61)
54
(1, 1)
(1, 1, 61)
(1, 61)
1
(1, 1)
(1, 1, 61)
(1, 61)
35
(1, 1)
(1, 1, 61)
(1, 61)
48
(1, 1)
(1, 1, 61)
(1, 61)
49
(1, 1)
(1, 1, 61)
(1, 61)
37
(1, 1)
(1, 1, 61)
(1, 61)
39
(1, 1)
(1, 1, 61)
(1, 61)
46
(1, 1)
(1, 1, 61)
(1, 61)
59
(1, 1)
(1